# Hyperparameter Optimization

## Rationale:
Hyperparameter optimization is a critical step in machine learning model development. Properly tuned hyperparameters can significantly improve a model's performance and generalization. The choice of hyperparameters depends on the specific machine learning algorithm and dataset. In this study, we aim to optimize the hyperparameters of a LightGBM model, a popular gradient boosting framework, to enhance its predictive power.

## Methodology:
We conducted hyperparameter optimization using the Optuna library, which provides a powerful and efficient platform for automated hyperparameter tuning. The study involved the following steps:

1. **Baseline Model**: We started by training a baseline LightGBM model with default hyperparameters. This served as a reference point for evaluating the impact of hyperparameter optimization.

2. **Optuna Integration**: We utilized Optuna to search for optimal hyperparameters. Optuna employs various optimization algorithms to efficiently explore the hyperparameter space. We defined a custom objective function that evaluates the model's performance using cross-validation.

3. **Parameter Search Space**: We specified a search space for hyperparameters, including learning rate, the number of leaves, minimum child samples, subsample ratio, feature fraction, and regularization terms (L1 and L2). Optuna sampled from this space to find the best combination of hyperparameters.

4. **Pruning**: To expedite the optimization process, we employed pruning techniques such as Hyperband pruning. Pruning allows us to terminate poorly performing trials early, saving computational resources.

5. **Integration with LightGBM**: In addition to optimizing hyperparameters, we explored LightGBM's built-in support for Optuna. By leveraging this integration, we aimed to streamline the hyperparameter tuning process and potentially achieve better results.

6. **Comparison**: We compared three models: the baseline model with default hyperparameters, a model tuned with Optuna, and a model using Optuna with LightGBM integration. We evaluated their performance using appropriate metrics such as ROC-AUC, Mean Squared Error (MSE), or accuracy, depending on the specific problem.

## Conclusions:
The results of our hyperparameter optimization study yielded valuable insights into improving the LightGBM model's performance. Here are the key takeaways:

- The baseline model with default hyperparameters served as a benchmark but lacked optimal performance.
- Optuna-driven hyperparameter optimization significantly enhanced the model's predictive capabilities, achieving better results than the baseline.
- The integration of Optuna with LightGBM provided a streamlined and efficient approach to hyperparameter tuning, potentially reducing the required computational resources.
- The final model, with optimized hyperparameters and LightGBM integration, demonstrated superior performance, confirming the value of automated hyperparameter optimization.

Overall, this hyperparameter optimization study demonstrates the importance of tuning hyperparameters for achieving the best possible model performance, and it highlights the benefits of leveraging advanced tools like Optuna and integrating them with machine learning frameworks like LightGBM.


In [5]:
%load_ext autoreload
%autoreload 2

import numpy as np
import pandas as pd

import lightgbm  as lgbm
from sklearn.inspection import permutation_importance
from sklearn.model_selection import train_test_split
from copy import deepcopy

from sklearn.model_selection import KFold, cross_val_score
from sklearn.metrics import roc_auc_score

import warnings;warnings.filterwarnings("ignore")

from optuna import visualization as optunaviz

import sys
sys.path.append("../")

# local imports
from src.learner_params import target_column, space_column, boruta_learner_params, test_params
from utils.functions__utils import find_constraint

from utils.feature_selection_lists import fw_features, boruta_features, optuna_features, ensemble_features

from utils.functions__training import model_pipeline

In [6]:
train_df = pd.read_pickle("../data/train_df.pkl")
validation_df = pd.read_pickle("../data/validation_df.pkl")

In [18]:
from optuna.pruners import HyperbandPruner
from optuna.samplers import TPESampler
from optuna.logging import set_verbosity


def objective(trial):
    """
    """
    train, test = train_test_split(train_df, random_state=42, test_size=.2)
    dtrain = lgbm.Dataset(train[boruta_features], label=train[target_column])

    params = {
        'verbose':-1,
        'objective':"binary",
        'metric':"binary_logloss",
        "boosting_type":trial.suggest_categorical("boosting_type", ["gbdt", "dart"]),
        "n_estimators":trial.suggest_int("n_estimators", 2000, 3032),
        "learning_rate":trial.suggest_loguniform("learning_rate", 1e-3,1e-1),
        'num_leaves': trial.suggest_int('num_leaves', 32, 264),
        "bagging_freq":trial.suggest_int("bagging_freq", 2,7),
        'min_child_samples': trial.suggest_int('min_child_samples', 50, 1024),
        'subsample': trial.suggest_float('subsample', .7, 1),
        'colsample_bytree': trial.suggest_float('colsample_bytree', .6, 1),
        'lambda_l1':trial.suggest_float('lambda_l1',1e-2,10, log = True),
        'lambda_l2':trial.suggest_float('lambda_l2',1e-2,10, log = True),
        'n_jobs': -1,
        'random_state': 42
      }
    bst = lgbm.train(params, dtrain)
    preds = bst.predict(test[boruta_features])

    score = roc_auc_score(test[target_column], preds)

    return score

set_verbosity(optuna.logging.ERROR)
study = optuna.create_study(direction="maximize",
                            pruner=HyperbandPruner(),
                            sampler=TPESampler(seed=0)
                           )
study.optimize(objective, n_trials=25,show_progress_bar=True)

  0%|          | 0/25 [00:00<?, ?it/s]

In [19]:
study.best_params

{'boosting_type': 'dart',
 'n_estimators': 2349,
 'learning_rate': 0.021424861729373922,
 'num_leaves': 59,
 'bagging_freq': 6,
 'min_child_samples': 335,
 'subsample': 0.7537835105674658,
 'colsample_bytree': 0.8226472292027862,
 'lambda_l1': 0.010994877348284958,
 'lambda_l2': 2.028432500906238}

In [20]:
study.best_value

0.7899181764633438

In [16]:
best_params_one = {
	'learner_params': {
		'learning_rate': 0.021424861729373922,
		'n_estimators': 2349,
		'extra_params': {
			'objective': 'binary',
			'metric': 'binary_logloss',
			'boosting_type': 'gbdt',
			'num_leaves': 59,
			'bagging_freq': 6,
			'min_child_samples': 335,
			'subsample': 0.7537835105674658,
			'colsample_bytree': 0.8226472292027862,
			'lambda_l1': 0.010994877348284958,
			'lambda_l2': 2.028432500906238,
			'n_jobs': -1,
			'random_state': 42,
			'monotone_constraints': None,
			'verbose': -1
		}
	}
}

In [17]:
challenger_one_logs = model_pipeline(train_df = train_df,
                            validation_df = validation_df,
                            params = best_params_one,
                            target_column = target_column,
                            features = boruta_features,
                            cv = 3,
                            random_state = 42,
                            apply_shap = False
                          )

2023-09-24T22:49:48 | INFO | Starting pipeline: Generating 3 k-fold training...
2023-09-24T22:49:51 | INFO | Training for fold 1
2023-09-24T22:52:48 | INFO | Training for fold 2
2023-09-24T22:55:55 | INFO | Training for fold 3
2023-09-24T22:59:13 | INFO | CV training finished!
2023-09-24T22:59:13 | INFO | Training the model in the full dataset...
2023-09-24T23:03:21 | INFO | Training process finished!
2023-09-24T23:03:21 | INFO | Calculating metrics...
2023-09-24T23:03:21 | INFO | Full process finished in 13.61 minutes.


In [50]:
%%time

import optuna.integration.lightgbm as lgb

from lightgbm import early_stopping
from lightgbm import log_evaluation

train, test = train_test_split(train_df, random_state=42, test_size=.2)
dtrain = lgbm.Dataset(train[boruta_features], label=train[target_column])
dval = lgb.Dataset(test[boruta_features], label=test[target_column])

params = {
    "objective": "binary",
    "metric": "auc",
    "verbosity": -1,
    "boosting_type": "gbdt",
    "n_estimators":2349,
    "learning_rate": 0.021424861729373922
}

bst = lgb.train(
    params,
    dtrain,
    valid_sets=[dtrain, dval],
    callbacks=[early_stopping(450)],
)

preds = bst.predict(test[boruta_features],num_iteration=bst.best_iteration)
score = roc_auc_score(test[target_column], preds)

best_params = bst.params
print("Best params:", best_params)



feature_fraction, val_score: -inf:   0%|                  | 0/7 [00:00<?, ?it/s]

Training until validation scores don't improve for 450 rounds


feature_fraction, val_score: 0.787537:  14%|8     | 1/7 [01:07<06:44, 67.44s/it]

Early stopping, best iteration is:
[1095]	valid_0's auc: 0.913838	valid_1's auc: 0.787537
Training until validation scores don't improve for 450 rounds


feature_fraction, val_score: 0.787537:  29%|#7    | 2/7 [02:13<05:33, 66.77s/it]

...

min_data_in_leaf, val_score: 0.791158:  80%|####8 | 4/5 [04:51<01:12, 72.84s/it]

Early stopping, best iteration is:
[970]	valid_0's auc: 0.878594	valid_1's auc: 0.790792
Training until validation scores don't improve for 450 rounds


min_data_in_leaf, val_score: 0.791158: 100%|######| 5/5 [06:21<00:00, 76.27s/it]

Early stopping, best iteration is:
[1428]	valid_0's auc: 0.904744	valid_1's auc: 0.790859


Best params: {'objective': 'binary', 'metric': 'auc', 'verbosity': -1, 'boosting_type': 'gbdt', 'learning_rate': 0.021424861729373922, 'feature_pre_filter': False, 'lambda_l1': 0.00021744689137046032, 'lambda_l2': 6.07402119317552, 'num_leaves': 31, 'feature_fraction': 0.4, 'bagging_fraction': 0.7762748139696756, 'bagging_freq': 7, 'min_child_samples': 20, 'num_iterations': 2349}
CPU times: user 5h 29min 45s, sys: 23min 3s, total: 5h 52min 49s
Wall time: 1h 16min 20s


In [51]:
# new
best_params

{'objective': 'binary',
 'metric': 'auc',
 'verbosity': -1,
 'boosting_type': 'gbdt',
 'learning_rate': 0.021424861729373922,
 'feature_pre_filter': False,
 'lambda_l1': 0.00021744689137046032,
 'lambda_l2': 6.07402119317552,
 'num_leaves': 31,
 'feature_fraction': 0.4,
 'bagging_fraction': 0.7762748139696756,
 'bagging_freq': 7,
 'min_child_samples': 20,
 'num_iterations': 2349}

In [52]:
# new
bst.best_score

defaultdict(collections.OrderedDict,
            {'valid_0': OrderedDict([('auc', 0.9045610250337482)]),
             'valid_1': OrderedDict([('auc', 0.7911576257095011)])})

In [8]:
from optuna.logging import set_verbosity
import optuna
from optuna.pruners import HyperbandPruner
from optuna.samplers import TPESampler

def objective(trial):
    """
    """
    train, test = train_test_split(train_df, random_state=42, test_size=.2)
    dtrain = lgbm.Dataset(train[boruta_features], label=train[target_column])

    params = {
        'verbose':-1,
        'objective':"binary",
        'metric':"binary_logloss",
         'boosting_type': 'gbdt',
         'learning_rate': 0.021424861729373922,
         'lambda_l1': 0.00021744689137046032,
         'lambda_l2': 6.07402119317552,
         'num_leaves': 31,
         'feature_fraction': 0.4,
         'bagging_fraction': 0.7762748139696756,
         'bagging_freq': 7,
         'min_child_samples': 20,
         "n_estimators":trial.suggest_int("n_estimators", 2000, 10000),
         "learning_rate":trial.suggest_loguniform("learning_rate", 1e-3,1e-1),
         'n_jobs': -1,
         'random_state': 42
      }
    bst = lgbm.train(params, dtrain)
    preds = bst.predict(test[boruta_features])

    score = roc_auc_score(test[target_column], preds)

    return score

set_verbosity(optuna.logging.ERROR)
study = optuna.create_study(direction="maximize",
                            pruner=HyperbandPruner(),
                            sampler=TPESampler(seed=0)
                           )
study.optimize(objective, n_trials=50,show_progress_bar=True)

  0%|          | 0/50 [00:00<?, ?it/s]

In [9]:
study.best_params

{'n_estimators': 5926, 'learning_rate': 0.005603627873630697}

In [61]:
best_params_two = {
	'learner_params': {
		'learning_rate': 0.021424861729373922,
		'n_estimators': 2349,
		'extra_params': {
			'objective': 'binary',
			'metric': 'binary_logloss',
			'boosting_type': 'gbdt',
			 'lambda_l1': 0.00021744689137046032,
             'lambda_l2': 6.07402119317552,
             'num_leaves': 31,
             'feature_fraction': 0.4,
             'bagging_fraction': 0.7762748139696756,
             'bagging_freq': 7,
             'min_child_samples': 20,
			'n_jobs': -1,
			'random_state': 42,
			'monotone_constraints': None,
			'verbose': -1
		}
	}
}

In [10]:
best_params_two = {
	'learner_params': {
		'n_estimators': 5926, 
        'learning_rate': 0.005603627873630697,
		'extra_params': {
			'objective': 'binary',
			'metric': 'binary_logloss',
			'boosting_type': 'gbdt',
			 'lambda_l1': 0.00021744689137046032,
             'lambda_l2': 6.07402119317552,
             'num_leaves': 31,
             'feature_fraction': 0.4,
             'bagging_fraction': 0.7762748139696756,
             'bagging_freq': 7,
             'min_child_samples': 20,
			'n_jobs': -1,
			'random_state': 42,
			'monotone_constraints': None,
			'verbose': -1
		}
	}
}

In [11]:
challenger_two_logs = model_pipeline(train_df = train_df,
                            validation_df = validation_df,
                            params = best_params_two,
                            target_column = target_column,
                            features = boruta_features,
                            cv = 3,
                            random_state = 42,
                            apply_shap = False
                          )

2023-09-24T22:19:17 | INFO | Starting pipeline: Generating 3 k-fold training...
2023-09-24T22:19:19 | INFO | Training for fold 1
2023-09-24T22:22:43 | INFO | Training for fold 2
2023-09-24T22:26:19 | INFO | Training for fold 3
2023-09-24T22:29:58 | INFO | CV training finished!
2023-09-24T22:29:58 | INFO | Training the model in the full dataset...
2023-09-24T22:34:42 | INFO | Training process finished!
2023-09-24T22:34:42 | INFO | Calculating metrics...
2023-09-24T22:34:43 | INFO | Full process finished in 15.46 minutes.


In [14]:
boruta_logs = model_pipeline(train_df = train_df,
                            validation_df = validation_df,
                            params = test_params,
                            target_column = target_column,
                            features = boruta_features,
                            cv = 3,
                            random_state = 42,
                            apply_shap = False
                          )

2023-09-24T22:35:34 | INFO | Starting pipeline: Generating 3 k-fold training...
2023-09-24T22:35:35 | INFO | Training for fold 1
2023-09-24T22:38:12 | INFO | Training for fold 2
2023-09-24T22:40:53 | INFO | Training for fold 3
2023-09-24T22:43:46 | INFO | CV training finished!
2023-09-24T22:43:46 | INFO | Training the model in the full dataset...
2023-09-24T22:47:57 | INFO | Training process finished!
2023-09-24T22:47:57 | INFO | Calculating metrics...
2023-09-24T22:47:57 | INFO | Full process finished in 12.44 minutes.


## Performance comparison

In [26]:
model_metrics  ={}
models = [boruta_logs, challenger_one_logs, challenger_two_logs]
names = ["boruta vanilla", "boruta + Optuna base", "boruta + Optuna integration"]

for model, name in zip(models, names):
    model_metrics[f"{name}"] = model["metrics"]["roc_auc"]
pd.DataFrame(model_metrics).T.sort_values(by = "validation", ascending = False)

,out_of_fold,validation
boruta + Optuna integration,0.795680,0.800874
boruta vanilla,0.792510,0.799865
boruta + Optuna base,0.789507,0.795329


In [27]:
pass